In [1]:
# Boilerplate imports

import numpy as np

import torch
from torch.utils.data import DataLoader

from tqdm.autonotebook import trange
import time

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)

import matplotlib.pyplot as plt

/var/folders/w7/dpsq9mj54t7gg3nc3923pc0r0000gn/T/ipykernel_11382/1112600444.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [2]:
# Load NIR graph and convert to a Rockpool torch module

import nir
from rockpool.nn.modules.torch.nir import from_nir

nirgraph = nir.read('braille_noDelay_noBias_subtract_subgraph.nir')
net = from_nir(nirgraph)
print(net)

GraphExecutor(
  (fc1): LinearTorch()
  (fc2): LinearTorch()
  (input): Identity()
  (lif1): LIFTorch()
  (lif2): LIFTorch()
  (output): Identity()
)


/Users/dylan/SynSense Dropbox/Dylan Muir/LiveSync/Development/Projects/nir/rockpool_nir/rockpool/nn/modules/torch/linear_torch.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(weight) if weight is not None else None,
/Users/dylan/SynSense Dropbox/Dylan Muir/LiveSync/Development/Projects/nir/rockpool_nir/rockpool/nn/modules/torch/torch_module.py:258: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  super().register_parameter(key, nn.Parameter(torch.tensor(value.data)))
/Users/dylan/SynSense Dropbox/Dylan Muir/LiveSync/Development/Projects/nir/rockpool_nir/rockpool/nn/modules/torch/nir.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor

In [3]:
# Impose a common `dt` on the LIF layers
dt = 1e-4
net.lif1.dt = dt
net.lif2.dt = dt

net._record = True

In [4]:
### Specify test data
test_data_path = "data/ds_test.pt"
ds_test = torch.load(test_data_path)
letter_written = ["Space", "A", "E", "I", "O", "U", "Y"]


In [34]:
### RUN TESTS
n_samples = len(ds_test)  # dataset size: 140
predicted_labels = []
actual_labels = []

for i in trange(n_samples):
    single_sample = next(iter(DataLoader(ds_test, batch_size=1, shuffle=True)))
    sample = single_sample[0].numpy()[0]  # shape: (256, 12)

    output, rec_dict = net(torch.tensor(sample))
    n_output_spikes = np.sum(output.squeeze().detach().numpy(), axis=0)

    predicted_label = int(np.argmax(n_output_spikes))
    actual_label = int(single_sample[1])
    # print("Predicted Label:", predicted_label)
    # print("Actual Label:   ", actual_label)
    predicted_labels.append(predicted_label)
    actual_labels.append(actual_label)

predicted_labels = np.array(predicted_labels)
actual_labels = np.array(actual_labels)
n_correct = np.count_nonzero(predicted_labels == actual_labels)
print(f"n_correct {n_correct} out of {n_samples} ({n_correct / n_samples * 100.}%)")

100%|██████████| 140/140 [00:13<00:00, 10.20it/s]

n_correct 108 out of 140 (77.14285714285715%)


In [35]:
# Save accuracy results and internal network activity
np.save('Rockpool_accuracy_subtract.npy', n_correct / n_samples)

test_sample = torch.load("data/ds_test.pt")[0][0]
_, rec_dict = net(torch.tensor(test_sample))

np.save('Rockpool_activity_subtract.npy', rec_dict.cache['lif1'].squeeze().detach().numpy())

/var/folders/w7/dpsq9mj54t7gg3nc3923pc0r0000gn/T/ipykernel_11382/1876152293.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, rec_dict = net(torch.tensor(test_sample))
